    O código a seguir ira executar o algoritmo SDES, que é uma versão simplificada do DES, focada no desenvolvimento para aprendizado. Irei explicar o funcionamento de função a função, afim de explicar o funcionamento geral do código.
    

In [1]:
def perm10(k): 
    chaveP = k[2]+k[4]+k[1]+k[6]+k[3]+k[9]+k[0]+k[8]+k[7]+k[5]
    
    print("Chave após a execução da função perm10:", chaveP)
    
    return chaveP

Aqui temos a primeira permutação da chave, onde a modificamos conforme o algoritmo pedia, com os caracteres nos seus devidos lugares. No final, fazemos a demonstração de como ficou a chave, antes do retorno do valor.


In [2]:
def ls(k, n): # Chave, numero do shift
        chave = ""
        
        changeList = [0 for i in range(len(k))]

        for i in range(len(k)):
            if (i - n) < 0:
                changeList[len(k) - n + i] = k[i]
            else:
                changeList[i - n] = k[i]
        
        for bit in changeList:
            chave += bit
        
        print("Chave após a execução da função de left shift:", chave)
        
        return chave

Aqui temos a função que faz o left shift dos caracteres. Ele pode modificar quantos caracteres a direita for determinado, como no algoritmo, que pede um shift de um e dois caracteres em diferentes momentos. Em seguida, temos o resultado após a execução da função.

In [3]:
def perm8(kLS):
    chaveP = kLS[5]+kLS[2]+kLS[6]+kLS[3]+kLS[7]+kLS[4]+kLS[9]+kLS[8]
    
    print("Chave após a execução da função perm8:", chaveP)
    
    return chaveP

A função acima faz uma permutação na chave, que modifica a ordem dos bits e retira dois deles. A seguir, mostra o resultado após a execução da função.

In [4]:
def genChave(chave): 

    # Primeira metade
    c1 = chave[0:5]
    # Segunda metade
    c2 = chave[5:10]

    c1 = ls(c1, 1)

    c2 = ls(c2, 1)

    k1 = c1 + c2
    k1 = perm8(k1)
    k2 = ls(c1, 2) + ls(c2, 2)
    k2 = perm8(k2)

    print("Chaves K1 e K2 criadas com a função de geração de chave:", k1, k2)

    return k1, k2

Aqui temos a primeira grande função. A função de geração de chave cria as subchaves K1 e K2, que serão utilizadas mais pra frente na encriptação da mensagem. Ela é criada usando a chave inicial, que é dividida em duas partes iguais(c1 e c2). C1 e C2 passam por modificações, que são feitas usando as funções de left shift de um caracter em cada. K1 se torna então a soma de C1 e C2. Logo após, C1 e C2 passam novamente por modificações, usando a função left shift, porém agora movendo dois bits ao invés de um. K2 e formado também com a soma de C1 e C2, após as devidas alterações.

In [5]:
def ip(msg):
    msg = msg[1] + msg[5] + msg[2] + msg[0] + msg[3] + msg[7] + msg[4] + msg[6]  

    print("Mensagem após a permutação inicial:", msg)
    
    return msg

A função acima faz uma permutação na mensagem, onde os bits são alterados conforme pede o algoritmo. Logo após, é mostrado o resultado após a execução da função.

In [6]:
def xor(a, b):
    saida = ""

    for i in range(len(a)):
        if a[i] == b[i]:
            saida += "0"
        else:
            saida += "1"
    return saida

A função acima faz o XOR padrão, onde quando os bits são iguais o valor de retorno é 0, e quando os valores são diferentes a saída é 1.

In [7]:
def F(right, subchave):
    exp = right[3] + right[0] + right[1] + right[2] + right[1] + right[2] + right[3] + right[0]
    
    print("Lado direito da mensagem após a expansão de bits:", exp)

    P = [[xor(exp[0], subchave[0]), xor(exp[1], subchave[1]), xor(exp[2], subchave[2]), xor(exp[3], subchave[3])],
    [xor(exp[4], subchave[4]), xor(exp[5], subchave[5]), xor(exp[6], subchave[6]), xor(exp[7], subchave[7])]]
    
    print("Matriz formada através de um XOR bit a bit entre a mensagem expandida e subchave de entrada:", P)

    S0 = [["01","00","11","10"],
          ["11","10","01","00"],
          ["00","10","01","11"],
          ["11","01","11","10"]]

    S1 = [["00","01","10","11"],
          ["10","00","01","11"],
          ["11","00","01","00"],
          ["10","01","00","11"]]
        
    msg = S0[int(P[0][0] + P[0][3], 2)][int(P[0][1] + P[0][2], 2)] + S1[int(P[1][0] + P[1][3], 2)][int(P[1][1] + P[1][2], 2)]

    msg = msg[1] + msg[3] + msg[2] + msg[0]
    
    print("Mensagem formada após a execução da função F:", msg)

    return msg

F é a função de maior complexidade no algoritmo. Ela recebe como parâmetro o lado direito da mensagem, e uma subchave, podendo ser K1 ou K2 dependendo do momento em que a função é utilizada. É feita uma expansão nos 4 caracteres da mensagem, onde é feita uma permutação junto com uma repetição de bits, conforme mostrado em *exp*. Logo após essa expansão/permutação, é criada a matriz *P*, que é formada através de um XOR bit a bit da *exp* e da subchave obtida na entrada. Agora precisamos escolher apenas 4 bits para ser a saída, para isso utilizamos as matrizes *P*, *S0* e *S1*, as famosas "Xbox :)". Utilizaremos os bits referentes as posições P[0][[0] e P[0][3] para decidir a linha a ser escolhida em S0, P[0][1] e P[0][2] para decidir a coluna. O mesmo para escolher o valor em S1, porém puxando a partir de P[1], com a coluna sendo a mesma que utilizada anteriormente. Após a seleção dos valores na matriz, é feita uma permutação para mudança de bits, logo depois é feito um print que mostra toda a mudança feita na função. Vale ressaltar que outros prints são feitos durante a função para demonstrar as mudanças passo a passo.


In [8]:
def feistel(left, right, subchave):
    resultado = F(right, subchave)

    resultado = xor(left, resultado)

    print("Resultado da mensagem após a rodada de Feistel:", resultado)

    return right, resultado

Acima temos Feistel, que é a função principal do algoritmo. As entradas da função são os lados esquerdo e direito da mensagem e a subchave que será utilizada em *F*. Primeiramente, F é executado no lado utilizando o lado direito e a subchave. Logo após, um *XOR* é executado entre o lado esquerdo da mensagem e o resultado de F. Aqui a função *Switch* não existe de maneira isolada, mas o switch é feito na saida da função, onde os valores são retorbnados em posições diferentes. Antes do retorno dos valores, é feito um print do valor após as alterações, lembrando que o algoritmo altera apenas 4 bits em uma rodada de Feistel.


In [9]:
def ipInv(msg):
    msg = msg[3] + msg[0] + msg[2] + msg[4] + msg[6] + msg[1] + msg[7] + msg[5]

    print("Mensagem formada após a permutação final:", msg)

    return msg


Acima temos a função de permutação final, onde é a ultima permutação na mensagem conforme o algoritmo pede. Vale ressaltar que Ip e inversivel, e a permutação final é a Inversa de IP.

In [10]:
def SDES(msg, chave):
    k1, k2 = genChave(perm10(chave))

    msg = ip(msg)

    L, R = msg[0:4], msg[4:8]

    L, R = feistel(L, R, k1)

    R, L = feistel(L, R, k2)

    resultado = ipInv(L + R)

    print("Encriptação: " + resultado)

    return resultado

Com todas as funções já explicadas, iremos para a função de encriptação. Aqui temos a função que executa todas as funções que são necessárias para a encriptação da mensagem. A função funciona de modo bem simples: logo de inicio, pegamos a chave escolhida e executamos perm10. A saída já vai direto para função de geração de chave, osde são retornados os valores para K1 e K2. Logo apos, executamos a permutação inicial sobre a mensagem em claro. dividimos a mensagem em lado L e R, sendo eles os lados esquerdo e direito sucessivamente. Logo após essa divisão, é passado os lados direito e esquerdo juntamente com a subchave K1 para a função de Feistel. A saída é coloca em L e R e novamente é feita outra rodada de Feistel, porém utilizando a chave K2 como argumento invés de K1. A saída da segunda rodada é colocada em R e L, para contornar o switch final que ocorre na função, já que essa troca não deve existir na última rodada. Por fim, é executada a permutação final sobre a mensagem, e o retorno já é a men sagem encriptada.

In [11]:
def DSDES(msg, chave):
    k1, k2 = genChave(perm10(chave))

    msg = ip(msg)

    L, R = msg[0:4], msg[4:8]

    L, R = feistel(L, R, k2)

    R, L = feistel(L, R, k1)

    resultado = ipInv(L + R)

    print("Descriptação: " + resultado)

Aqui temos o código de descriptografia do SDES. Para evitar a repetição de informações, a função DSDES é basicamente a mesma coisa que a função de encriptação, mudando somente a ordem das subchaves utilizadas em Feistel. (Tirei o retorno fi

In [12]:
DSDES(SDES("11010111", "1010000010"), "1010000010")


Chave após a execução da função perm10: 1000001100
Chave após a execução da função de left shift: 00001
Chave após a execução da função de left shift: 11000
Chave após a execução da função perm8: 10100100
Chave após a execução da função de left shift: 00100
Chave após a execução da função de left shift: 00011
Chave após a execução da função perm8: 01000011
Chaves K1 e K2 criadas com a função de geração de chave: 10100100 01000011
Mensagem após a permutação inicial: 11011101
Lado direito da mensagem após a expansão de bits: 11101011
Matriz formada através de um XOR bit a bit entre a mensagem expandida e subchave de entrada: [['0', '1', '0', '0'], ['1', '1', '1', '1']]
Mensagem formada após a execução da função F: 1111
Resultado da mensagem após a rodada de Feistel: 0010
Lado direito da mensagem após a expansão de bits: 00010100
Matriz formada através de um XOR bit a bit entre a mensagem expandida e subchave de entrada: [['0', '1', '0', '1'], ['0', '1', '1', '1']]
Mensagem formada após a

Aqui temos a execução de ambas as funções, tanto a de criptografia quanto a de descriptografia. Fiz diversos prints durante as funções para que seja possível observar o que cada função faz sobre as informações passo a passo. Com isso finalizo a apresentação do algoritmo, com a demonstração de como ele funciona.